# CNN_Daily_Mail_Final_Model_4a.1_Bert_Based_Transformer_Cosine_Extractive_Summarization
    Nov 6, 2022


This notebook has the following model built for an extractive summarizer based on Bert Sentence Transfomer Model from Hugging Face.

1. **Bert based sentence transformer** is built from scratch with a non-linear activation.
2.SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. This framework can be used to compute sentence / text embeddings for more than 100 languages. **These** embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.

Dataset Summary

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering. 

Data Fields

    id: a string containing the heximal formated SHA1 hash of the url where the story was retrieved from
    article: a string containing the body of the news article
    highlights: a string containing the highlight of the article as written by the article author
    
Model Details

    Sentence Transformer: Sentence Transformer
    Pre-Training: bert-base-uncased
    **Supervised**: Unsupervised Model
    Classification: Degree Centrality of Sentences in a Document
    Trigram Blocking: Yes
    Fine Tuning: None
    Evaluation Metrics: RougeL and Cosine Similarity

    ## Step 1: use an existing language model
    word_embedding_model = models.Transformer('bert-base-uncased')

    ## Step 2: use a pool function over the token embeddings
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode_mean_tokens=True)

    ## Join steps 1 and 2 using the modules argument
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=pooling_model.get_sentence_embedding_dimension(), activation_function=nn.ReLU())
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
    


# 1. Setup

#### This section install key libraries

In [ ]:
!pip install bert-extractive-summarizer --quiet

     |████████████████████████████████| 5.5 MB 15.4 MB/s 
     |████████████████████████████████| 163 kB 58.0 MB/s 
     |████████████████████████████████| 7.6 MB 50.9 MB/s 


In [ ]:
!pip install datasets --quiet
!pip install nltk --quiet

     |████████████████████████████████| 441 kB 14.3 MB/s 
     |████████████████████████████████| 95 kB 2.0 MB/s 
     |████████████████████████████████| 115 kB 59.4 MB/s 
     |████████████████████████████████| 212 kB 57.8 MB/s 
     |████████████████████████████████| 127 kB 48.0 MB/s 
     |████████████████████████████████| 115 kB 53.8 MB/s 


In [ ]:
!pip install -q rouge_score

In [ ]:
!pip install -q evaluate


     |████████████████████████████████| 72 kB 1.1 MB/s 


In [ ]:
!pip install -U spacy
!pip install -U spacy-lookups-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98.5 MB 1.3 MB/s 


In [ ]:
!pip install -r rouge/requirements.txt
!pip install rouge-score
!pip install rouge_score

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_lg

2022-11-06 16:02:04.462876: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# 2.0 Import Libraries

In [ ]:
# NLTK
import re # relugar expression
import nltk # natural language toolkit for sentence tokenization and display
import string
import heapq
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.util import ngrams
import evaluate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import os
import logging

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
import evaluate

In [ ]:
import spacy
import pandas as pd

print(spacy.__version__)
print(pd.__version__)

3.4.2
1.3.5


In [ ]:
import pickle
import subprocess
import sys
import nltk
from nltk import Nonterminal, nonterminals, Production, CFG, PCFG

In [ ]:
#shift reduce parser example
from nltk.grammar import Nonterminal
from nltk.parse.api import ParserI
from nltk.tree import Tree

In [ ]:
nlp = spacy.load("en_core_web_lg")


In [ ]:
from rouge_score import rouge_scorer

In [ ]:
import time

In [ ]:
# Mount drive for saving model checkpoints, loading Task 2 data below

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3.0 Load the dataset



In [ ]:
from datasets import load_dataset, load_metric


In [ ]:
# Load 1000 examples of CNN Dailymail from a CSV File

filepath = 'drive/My Drive/Colab_Notebooks_1/cnn_dailymail_1000.csv'
dataset = load_dataset('csv', data_files = filepath, split='train' )
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d8ef26a734f1246a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Dataset({
    features: ['document', 'summary'],
    num_rows: 1000
})

In [ ]:
dataset['summary'][0]

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

# 4.0 Helper Functions

In [ ]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.replace('\n', ' ')
    sentence = sentence.replace('?</s>', '?.')
    sentence = sentence.replace('</s>', '')
    sentence = sentence.replace("\'", "'")
    sentence = sentence.replace('--', '')
    sentence = sentence.replace('|', '')
    sentence = sentence.replace('/', '')
    sentence = sentence.replace('Dr.', 'Dr')
    sentence = sentence.replace('?.', '?. ')
    #sentence = sentence.replace('.', '. ')
    sentence = sentence.replace('!', '!. ')
    #sentence = sentence.replace('"', '')
    sentence = sentence.replace("This material may not be published, broadcast, rewritten, or redistributed.",'')
    #sentence = sentence.replace(' "', '')
    #sentence = sentence.replace('" ', '')
    return sentence

In [ ]:
def calculate_rouge(reference, predictions):
  rouge = evaluate.load('rouge')
  predictions = extracted_sentence_list
  references = orig_highlights_list
  results = rouge.compute(predictions=predictions,
                        references=references)
  print(results)

In [ ]:
"""
LexRank implementation
Source: https://github.com/crabcamp/lexrank/tree/dev
"""

import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.special import softmax
import logging

logger = logging.getLogger(__name__)

def degree_centrality_scores(
    similarity_matrix,
    threshold=None,
    increase_power=True,
):
    if not (
        threshold is None
        or isinstance(threshold, float)
        and 0 <= threshold < 1
    ):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None',
        )

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(
            similarity_matrix,
            threshold,
        )

    scores = stationary_distribution(
        markov_matrix,
        increase_power=increase_power,
        normalized=False,
    )

    return scores


def _power_method(transition_matrix, increase_power=True, max_iter=10000):
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    for _ in range(max_iter):
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)

    logger.warning("Maximum number of iterations for power method exceeded without convergence!")
    return eigenvector_next


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    # normalize probability distribution differently if we have negative transition values
    if np.min(weights_matrix) <= 0:
        return softmax(weights_matrix, axis=1)

    return weights_matrix / row_sum


def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = np.zeros(weights_matrix.shape)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1

    return create_markov_matrix(discrete_weights_matrix)


def stationary_distribution(
    transition_matrix,
    increase_power=True,
    normalized=True,
):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= n_1

    return distribution

In [ ]:
# Trigram Blocking for Extractive Summarization

def trigram_blocking(input_text):

  trigram_master_list=[]
  trigram_temp_list=[]
  clean_list = []
  trigram_flag = []
  output_list = input_text

  #output_list = ['oh my god', 'oh my god' ,'lovely day today', 'This year is good', 'Oh my god' ]

  for idx, sentence in enumerate(output_list):
    if idx == 0:
      token = nltk.word_tokenize(sentence.lower())  #tokenize your text and make it lowercase in onestep
      trigrams=ngrams(token,3)          # find all the trigram in text1
      #print(idx, sentence)
      for item in trigrams:
        #print(item)
        if item not in trigram_master_list:
          trigram_master_list.append(item)
      clean_list.append(sentence)
    elif idx > 0:
      token = nltk.word_tokenize(sentence.lower())  #tokenize your text and make it lowercase in onestep
      trigrams=ngrams(token,3)
      #print(idx, sentence) 
      for item in trigrams:
        #print(item)
        trigram_temp_list.append(item)
      trigram_flag = []  
      for trigram_temp in trigram_temp_list:
        if trigram_temp in trigram_master_list:
          trigram_flag.append("Y")
        else:
          trigram_flag.append("N")
      if "N" in trigram_flag:
        clean_list.append(sentence)
        for i in trigram_temp_list:
          trigram_master_list.append(i)

  #print(trigram_master_list)
  #print(clean_list)

  return (clean_list)

#5.0 Create a Dataset for Model Building and Evaluation
 

In [ ]:
# Generate a dataset of "x" examples for model evaulation
size_of_dataset = 1000 # change the value as you develop
small_dataset = dataset[0:size_of_dataset]

#6.0 Model Use Pre-Trained Bert for Extrative Summarization

In [ ]:
# Build the Model

!pip install -U sentence-transformers

import nltk
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.cluster import AgglomerativeClustering

from sentence_transformers import SentenceTransformer, models
from torch import nn

## Step 1: use an existing language model
word_embedding_model = models.Transformer('bert-base-uncased')
#word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)

## Step 2: use a pool function over the token embeddings
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),pooling_mode_mean_tokens=True)

## Join steps 1 and 2 using the modules argument
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

#dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=pooling_model.get_sentence_embedding_dimension(), activation_function=nn.ReLU())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 1.3 MB 74.3 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c4783dc3aff704c274a091735826f28bc8238139aab0c4138e89c046b9ae4653
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

#7.0 Evaluate Rouge Across Eval Data Set

In [ ]:
# RogueL Scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge = evaluate.load('rouge')

In [ ]:
# Iterate through the dataset, extract the summary and compute RougeL, Cosine Similarity Scores

start = time.time()

# get article from dataset
input_article = small_dataset['document'][0:size_of_dataset]
#len(input_article)

# get summary from dataset
input_highlights = small_dataset['summary'][0:size_of_dataset]
#len(input_highlights)

# zip article and summary
zipped_input = zip(input_article, input_highlights)

# Empty List to Store Scores
rougeL_precision = []
rougeL_recall = []
rougeL_fmeasure = []

rouge_1_list = []
rouge_2_list = []
rouge_L_list = []

cosine_similarity_results = []

# Counter for Tracking Results
count = 1

# Extracted Highlights
output_highlights_list = []

# iterate Through the Eval Dataset
for input_article, input_highlights in zipped_input:
  
  print('Example:', count)

  # Tokenize the string texts
  
  source_article_list = nltk.sent_tokenize(input_article)
  source_highlight_list = nltk.sent_tokenize(input_highlights)
  
# Join the list into a string
  formatted_source_article = " ".join(source_article_list)
  formatted_source_highlight = " ".join(source_highlight_list)
  
# get embeddings from model
  embeddings = model.encode(source_article_list, convert_to_tensor=True)

  output_list=[]

#Compute the pair-wise cosine similarities
  cos_scores = util.cos_sim(embeddings, embeddings).numpy()

#Compute the centrality for each sentence
  centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

#We argsort so that the first element is the sentence with the highest score
  most_central_sentence_indices = np.argsort(-centrality_scores)

  num_sentences = int(len(source_article_list) * 0.30) # 30% compression ration

  for idx in most_central_sentence_indices[0:num_sentences]:
    #print(source_article_list[idx].strip())
    output_list.append(source_article_list[idx])

  result = output_list 

# note result is a string returned by model
  summary = "".join(result)

# Take the string from summary and convert to list of strings for each sentence
  extracted_sentence_list = nltk.sent_tokenize(summary)

# Apply Trigram Blocking
  result_list = trigram_blocking(extracted_sentence_list)
  summary = " ".join(result_list)
  output_highlights_list.append(summary)

  predictions = " ".join(result_list)
  references = formatted_source_highlight

  rougeL_scores = scorer.score(predictions,
                      references)
  
  pred = [predictions]
  ref = [references]

  rouge_results = rouge.compute(predictions=pred, references=ref)
  print(rouge_results)

  rouge_1_score = (rouge_results['rouge1'])
  rouge_2_score = (rouge_results['rouge2'])
  rouge_L_score = (rouge_results['rougeL'])

  rouge_1_list.append(rouge_1_score)
  rouge_2_list.append(rouge_2_score)
  rouge_L_list.append(rouge_L_score)

  precision = (rougeL_scores['rougeL'].precision)
  recall = (rougeL_scores['rougeL'].recall)
  fmeasure = (rougeL_scores['rougeL'].fmeasure)

  rougeL_precision.append(precision)
  rougeL_recall.append(recall)
  rougeL_fmeasure.append(fmeasure)

  # calculate cosine similarity
  doc1 = nlp((" ".join(predictions)))
  doc2 = nlp((" ".join(references)))
  cosine_similarity = doc1.similarity(doc2)
  cosine_similarity_results.append(cosine_similarity)

  count = count + 1

print('\nTime:', time.time() - start)


Example: 1
{'rouge1': 0.1992619926199262, 'rouge2': 0.1412639405204461, 'rougeL': 0.13284132841328414, 'rougeLsum': 0.13284132841328414}
Example: 2
{'rouge1': 0.17270194986072426, 'rouge2': 0.05602240896358543, 'rougeL': 0.11699164345403901, 'rougeLsum': 0.14484679665738162}
Example: 3
{'rouge1': 0.1246458923512748, 'rouge2': 0.056980056980056974, 'rougeL': 0.08498583569405099, 'rougeLsum': 0.1019830028328612}
Example: 4
{'rouge1': 0.16666666666666669, 'rouge2': 0.033707865168539325, 'rougeL': 0.09999999999999999, 'rougeLsum': 0.09999999999999999}
Example: 5
{'rouge1': 0.09200968523002422, 'rouge2': 0.019464720194647202, 'rougeL': 0.06295399515738498, 'rougeLsum': 0.06295399515738498}
Example: 6
{'rouge1': 0.15584415584415584, 'rouge2': 0.07310704960835508, 'rougeL': 0.09350649350649351, 'rougeLsum': 0.11428571428571427}
Example: 7
{'rouge1': 0.1225382932166302, 'rouge2': 0.02637362637362637, 'rougeL': 0.07439824945295405, 'rougeLsum': 0.07439824945295405}
Example: 8
{'rouge1': 0.33149

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:111: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


{'rouge1': 0.07964601769911503, 'rouge2': 0.02222222222222222, 'rougeL': 0.05752212389380531, 'rougeLsum': 0.05752212389380531}
Example: 296
{'rouge1': 0.25217391304347825, 'rouge2': 0.14035087719298245, 'rougeL': 0.17391304347826086, 'rougeLsum': 0.17391304347826086}
Example: 297
{'rouge1': 0.09404388714733543, 'rouge2': 0.025236593059936908, 'rougeL': 0.06896551724137931, 'rougeLsum': 0.06896551724137931}
Example: 298
{'rouge1': 0.12075471698113208, 'rouge2': 0.03787878787878787, 'rougeL': 0.07169811320754717, 'rougeLsum': 0.07169811320754717}
Example: 299
{'rouge1': 0.1335952848722986, 'rouge2': 0.03944773175542406, 'rougeL': 0.08644400785854617, 'rougeLsum': 0.08644400785854617}
Example: 300
{'rouge1': 0.29493087557603687, 'rouge2': 0.14883720930232555, 'rougeL': 0.14746543778801843, 'rougeLsum': 0.14746543778801843}
Example: 301
{'rouge1': 0.2676056338028169, 'rouge2': 0.1142857142857143, 'rougeL': 0.1971830985915493, 'rougeLsum': 0.1971830985915493}
Example: 302
{'rouge1': 0.1040

In [ ]:
# Print Compute Time
print('\nTime:', time.time() - start)


Time: 2925.8387172222137


In [ ]:
# Clean up the extracted list
output_highlights_list = list(((map(preprocess, output_highlights_list))))

In [ ]:
# Export extractive summary to a CSV

# get article from dataset
input_article_list = small_dataset['document'][0:size_of_dataset]

# get summary from dataset
input_highlights_list = small_dataset['summary'][0:size_of_dataset]

df = pd.DataFrame(list(zip(input_article_list, input_highlights_list, output_highlights_list)),
                  columns = ['orig_article', 'orig_summary', 'extracted_summary'])

# Edit this filepath to wherever you saved the data in your Drive
filepath = 'drive/My Drive/Colab_Notebooks_1/model_4a1_extracted_CNNDaily1000.csv'

df.to_csv(filepath,index = False)

In [ ]:
# read back the csv file
data_import = pd.read_csv(filepath)        
#data_import.rename(columns = {'0':'orig_article', '1':'orig_summary', '2':'extracted_summary'}, inplace = True)

col1 = data_import.orig_article.values.tolist()
col2 = data_import.orig_summary.values.tolist()
col3 = data_import.extracted_summary.values.tolist()

In [ ]:
# Validate data_import
print(col1[0])
print("")

print(col2[0])
print("")

print(col3[0])

LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [ ]:
# Calculate Mean Rouge for Dataset

print("RougeL Precision Scores")
print(rougeL_precision)
print(len(rougeL_precision))
print(np.mean(np.asarray(rougeL_precision)))
print("")

print("RougeL Recall Scores")
print(rougeL_recall)
print(len(rougeL_precision))
print(np.mean(np.asarray(rougeL_recall)))
print("")

print("RougeL Fmeasure Scores")
print(rougeL_fmeasure)
print(len(rougeL_fmeasure))
print(np.mean(np.asarray(rougeL_fmeasure)))
print("")

print("Rouge 1 Scores")
print(rouge_1_list)
print(len(rouge_1_list))
print(np.mean(np.asarray(rouge_1_list)))
print("")

print("Rouge 2 Scores")
print(rouge_2_list)
print(len(rouge_2_list))
print(np.mean(np.asarray(rouge_2_list)))
print("")

print("Rouge L Scores")
print(rouge_L_list)
print(len(rouge_L_list))
print(np.mean(np.asarray(rouge_L_list)))
print("")


RougeL Precision Scores
[0.46153846153846156, 0.4489795918367347, 0.3902439024390244, 0.375, 0.34146341463414637, 0.4186046511627907, 0.3829787234042553, 0.5, 0.45454545454545453, 0.32142857142857145, 0.6486486486486487, 0.3055555555555556, 0.2222222222222222, 0.2903225806451613, 0.2608695652173913, 0.40816326530612246, 0.15384615384615385, 0.2619047619047619, 0.4222222222222222, 0.5714285714285714, 0.37209302325581395, 0.48148148148148145, 0.2608695652173913, 0.21621621621621623, 0.45454545454545453, 0.3125, 0.4722222222222222, 0.3, 0.34285714285714286, 0.39473684210526316, 0.5111111111111111, 0.3829787234042553, 0.5161290322580645, 0.43243243243243246, 0.4716981132075472, 0.3783783783783784, 0.4222222222222222, 0.39215686274509803, 0.5581395348837209, 0.3191489361702128, 0.4772727272727273, 0.41935483870967744, 0.43137254901960786, 0.5918367346938775, 0.40540540540540543, 0.46511627906976744, 0.5, 0.21951219512195122, 0.38095238095238093, 0.45454545454545453, 0.5476190476190477, 0.40

In [ ]:
# Calculate Mean Cosine Similarity

print("Cosine Similarity")
print(np.mean(np.asarray(cosine_similarity_results)))
print("")


Cosine Similarity
0.9919879761998597



In [ ]:
dataset = load_dataset('csv', data_files = filepath, split='train' )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-43dc9465c9a9be66/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['orig_article', 'orig_summary', 'extracted_summary'],
    num_rows: 1000
})